In [1]:
import pandas as pd
dataframe = pd.read_csv('noticias.csv', encoding = 'latin1')
dataframe.head()

,title,text,label
0,El Ministerio de Defensa no suspendió la mesad...,Previo a la reciente movilización de militares...,FALSO
1,Publicación sobre Cartel del COVID-19 incluy...,Atención denuncian de un Cartel del COVID-19...,FALSO
2,La sensación de agobio que a veces pueden prov...,Este artículo fue publicado originalmente por ...,FALSO
3,"Exguerrilleros de Farc son, en su mayoría, cam...","El senador del Partido Farc, Carlos Antonio Lo...",VERDADERO
4,Cuidado: cadena que ofrece pruebas COVID grati...,De nuevo y a través de la aplicación de mensaj...,FALSO


In [2]:
x = dataframe['text']
y = dataframe['label']

In [3]:
x

0       Previo a la reciente movilización de militares...
1       Atención denuncian de un Cartel del COVID-19...
2       Este artículo fue publicado originalmente por ...
3       El senador del Partido Farc, Carlos Antonio Lo...
4       De nuevo y a través de la aplicación de mensaj...
                              ...                        
5839                         Messi es hermano de Don Omar
5840    El Cadiz jugará un partido amistoso por el Mág...
5841           El proximo Miss Universo será en Guatemala
5842                        Ricardo Arjona es salvadoreño
5843                          King Flyp nació en Usulutan
Name: text, Length: 5844, dtype: object

In [4]:
y

0           FALSO
1           FALSO
2           FALSO
3       VERDADERO
4           FALSO
          ...    
5839        FALSO
5840    VERDADERO
5841        FALSO
5842        FALSO
5843        FALSO
Name: label, Length: 5844, dtype: object

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

2549    VERDADERO
2929    VERDADERO
3540        FALSO
2303    VERDADERO
1804        FALSO
          ...    
4931        FALSO
3264    VERDADERO
1653        FALSO
2607        FALSO
2732        FALSO
Name: label, Length: 4675, dtype: object

In [7]:
y_train

2549    VERDADERO
2929    VERDADERO
3540        FALSO
2303    VERDADERO
1804        FALSO
          ...    
4931        FALSO
3264    VERDADERO
1653        FALSO
2607        FALSO
2732        FALSO
Name: label, Length: 4675, dtype: object

In [8]:
import nltk
from nltk.corpus import stopwords

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\roroe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
spanish_stopwords = stopwords.words('spanish')

In [11]:
tfvect = TfidfVectorizer(stop_words=spanish_stopwords,max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

In [12]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [13]:
y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 83.58%


In [14]:
cf = confusion_matrix(y_test,y_pred, labels=['FALSO','VERDADERO'])
print(cf)

[[669  88]
 [104 308]]


In [15]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [16]:
fake_news_det('Atención denuncian de un Cartel del COVID-19. Con estas palabras se inicia un post de Facebook que se ha viralizado durante la pandemia por el nuevo coronavirus en Colombia. La publicación, que se ha compartido más de 38.000 veces, alerta a la ciudadanía')

['FALSO']


In [17]:
fake_news_det('El senador del Partido Farc, Carlos Antonio Lozada, aseguró en su cuenta de Twitter que “los firmantes del acuerdo de paz en su mayoría son campesinos, lo que saben hacer es cultivar la tierra, algo que justamente no tienen”.')

['VERDADERO']


In [18]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [19]:
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [20]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [21]:
fake_news_det1('El senador del Partido Farc, Carlos Antonio Lozada, aseguró en su cuenta de Twitter que “los firmantes del acuerdo de paz en su mayoría son campesinos, lo que saben hacer es cultivar la tierra, algo que justamente no tienen”.')

['VERDADERO']


In [22]:
fake_news_det1('El 1 de julio, Carlos Mario Marín, alcalde de Manizales, publicó en su cuenta de Twitter un trino en el que afirmó que “Manizales es una de las 9 ciudades en las que NO se encontró sobrecostos en la contratación para el manejo de la pandemia”.')

['VERDADERO']
